# check oof df

In [15]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


# make oof fn

In [16]:
def pred_fn(CFG, model, valid_loader):
    model.eval()

    valid_predictions = {"class_preds": np.empty(0)}
    valid_targets = {"class_targets": np.empty(0)}
    valid_input_info = {
        "series_date_key": [],
        "start_step": [],
        "end_step": []
    }

    for batch_idx, (inputs, targets, input_info_dict) in enumerate(valid_loader):
        inputs = inputs.to(CFG.device, non_blocking=True).float()
        targets = targets.to(CFG.device, non_blocking=True).float()
        with torch.no_grad():
            preds = model(inputs)
            # preds = torch.sigmoid(preds)
        valid_predictions = get_valid_values_dict(preds,
                                                  valid_predictions,
                                                  mode="preds")
        valid_targets = get_valid_values_dict(targets,
                                              valid_targets,
                                              mode="targets")
        valid_input_info = concat_valid_input_info(valid_input_info,
                                                   input_info_dict)

    del inputs, preds, targets
    gc.collect()
    torch.cuda.empty_cache()
    return valid_predictions, valid_targets, valid_input_info,


In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)


In [18]:
output_dir = "/kaggle/working/"
# exp_name = "exp012_targetdownsample_epoch10"
exp_name = "exp012_targetdownsample_epoch1"
exp_dir = os.path.join(output_dir, exp_name)
all_series_df_path = "/kaggle/input/targetdownsample_train_series_fold.parquet"

CFG = yaml.safe_load(open(os.path.join(output_dir, exp_name, "config.yaml"), "r"))

CFG = argparse.Namespace(**CFG)
print(CFG)

all_series_df = pd.read_parquet(all_series_df_path)


Namespace(T_0=1, T_mult=1, ave_kernel_size=301, batch_size=64, class_loss_weight=1.0, class_output_channels=1, competition_dir='/kaggle/input/child-mind-institute-detect-sleep-states', competition_name='dss', device='cuda', embedding_base_channels=16, eta_min=1e-09, event_df='/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv', event_loss_weight=1.0, event_output_channels=2, exp_category='baseline', exp_dir='/kaggle/working/exp012_targetdownsample_epoch1', exp_name='exp012_targetdownsample_epoch1', folds=[0, 1, 2, 3, 4], group_key='series_id', input_channels=10, input_dir='/kaggle/input', key_df='/kaggle/input/datakey_unique_non_null.csv', logger_path='/kaggle/working/exp012_targetdownsample_epoch1/train.log', lr=0.001, maxpool_kernel_size=11, model_type='target_downsample', n_epoch=1, n_folds=5, num_workers=2, output_channels=2, output_dir='/kaggle/working', print_freq=50, pseudo_weight_exp='exp003', seed=42, series_df='/kaggle/input/targetdownsample_train_series_

In [19]:
import time
def get_downsample_oof_df(
    valid_input_info_dict: dict,
    valid_preds_dict: dict,
    valid_targets_dict: dict,
    oof_df_fold: pd.DataFrame,
    config,
) -> pd.DataFrame:
    start_time = time.time()
    print("creating oof_df", end=" ... ")
    if "class_pred" in oof_df_fold.columns:
        oof_df_fold = oof_df_fold.drop(["class_pred"], axis=1)
    if "class_target" in oof_df_fold.columns:
        oof_df_fold = oof_df_fold.drop(["class_target"], axis=1)
    series_date_key_list = []
    class_pred_list, class_target_list, steps_list = [], [],  []
    for idx, (series_date_key, start_step, end_step) in enumerate(
            zip(
                valid_input_info_dict["series_date_key"],
                valid_input_info_dict["start_step"],
                valid_input_info_dict["end_step"],
            )):
        # preds targets shape: [batch, ch, data_length]
        class_pred = valid_preds_dict["class_preds"][idx]
        class_target = valid_targets_dict["class_targets"][idx]
        steps = range(start_step, end_step + 1, 12)
        series_date_data_num = len(steps)
        if series_date_data_num < len(class_pred[0]):
            class_pred = class_pred[0, :series_date_data_num]
            class_target = class_target[0, :series_date_data_num]
        elif series_date_data_num > len(class_pred[0]):
            padding_num = series_date_data_num - len(class_pred[0])
            class_pred = np.concatenate(
                [class_pred[0], -1 * np.ones(padding_num)], axis=0)
            class_target = np.concatenate(
                [class_target[0], -1 * np.ones(padding_num)], axis=0)
        else:
            class_pred = class_pred[0]
            class_target = class_target[0]
        if not (len(class_pred) == len(class_target)) or not (len(class_pred)
                                                              == len(steps)):
            print("len(class_pred)", len(class_pred))
            print("len(class_target)", len(class_target))
            print("len(steps)", len(steps))
            raise ValueError("preds and targets length is not same")
        class_pred_list.extend(class_pred)
        class_target_list.extend(class_target)
        steps_list.extend(steps)
        series_date_key_list.extend([series_date_key] * len(steps))
    oof_pred_target_df = pd.DataFrame({
        "series_date_key": series_date_key_list,
        "step": steps_list,
        "class_pred": class_pred_list,
        "class_target": class_target_list,
    })
    merge_start_time = time.time()
    print("merging oof_df")
    oof_df_fold = pd.merge(oof_df_fold,
                            oof_pred_target_df,
                            on=["series_date_key", "step"],
                            how="left")
    merge_elapsed = int(time.time() - merge_start_time) / 60
    print("merge elapsed time: {:.2f} min".format(merge_elapsed))
    elapsed = int(time.time() - start_time) / 60
    print(f" >> oof_df created. elapsed time: {elapsed:.2f} min")
    return oof_df_fold


In [20]:
for fold in CFG.folds:
    series_df = all_series_df[all_series_df["fold"] == fold].reset_index(drop=True)
    key_df = get_key_df(series_df)
    oof_df_fold = series_df.copy()
    init_cols = [
        "class_pred",
        "class_target",
    ]
    oof_df_fold = oof_df_fold.assign(
        **{col: -1 * np.ones(len(oof_df_fold))
            for col in init_cols})


    print(f"-- fold{fold} inference start --")
    # set model & learning fn
    model = get_model(CFG)
    model_path = os.path.join(exp_dir, f"fold{fold}_model.pth")
    print("model loading", model_path)
    model.load_state_dict(torch.load(model_path))
    model = model.to(CFG.device)
    # separate train/valid data
    infer_loader = get_loader(CFG, key_df, series_df, mode="infer")

    valid_predictions, valid_targets, input_info_dict_list = pred_fn(CFG, model, infer_loader)
    oof_df_fold = get_downsample_oof_df(
            input_info_dict_list,
            valid_predictions,
            valid_targets,
            oof_df_fold,
            CFG,
        )
    oof_df_fold = oof_df_fold[["series_id", "step", "minute", "second", "class_pred", "class_target"]]
    display(oof_df_fold.head())
    display(oof_df_fold[oof_df_fold["second"]==0].head())
    oof_df_fold = oof_df_fold.fillna(-1)
    # break
    oof_dir = os.path.join(CFG.output_dir, "_oof", CFG.exp_name)
    os.makedirs(oof_dir, exist_ok=True)
    oof_df_fold_path = os.path.join(oof_dir, f"raw_oof_df_fold{fold}.parquet")
    print("save oof_df to ", oof_df_fold_path)
    oof_df_fold.to_parquet(oof_df_fold_path)


-- fold0 inference start --
model loading /kaggle/working/exp012_targetdownsample_epoch1/fold0_model.pth
creating oof_df ... merging oof_df
merge elapsed time: 0.08 min
 >> oof_df created. elapsed time: 0.13 min


,series_id,step,minute,second,class_pred,class_target
0,03d92c9f6f8a,5760.0,0,0,0.064265,0.0
1,03d92c9f6f8a,5761.0,0,5,NaN,NaN
2,03d92c9f6f8a,5762.0,0,10,NaN,NaN
3,03d92c9f6f8a,5763.0,0,15,NaN,NaN
4,03d92c9f6f8a,5764.0,0,20,NaN,NaN


,series_id,step,minute,second,class_pred,class_target
0,03d92c9f6f8a,5760.0,0,0,0.064265,0.0
12,03d92c9f6f8a,5772.0,1,0,0.041327,0.0
24,03d92c9f6f8a,5784.0,2,0,0.037149,0.0
36,03d92c9f6f8a,5796.0,3,0,0.027678,0.0
48,03d92c9f6f8a,5808.0,4,0,0.039245,0.0


save oof_df to  /kaggle/working/_oof/exp012_targetdownsample_epoch1/raw_oof_df_fold0.parquet
-- fold1 inference start --
model loading /kaggle/working/exp012_targetdownsample_epoch1/fold1_model.pth
creating oof_df ... merging oof_df
merge elapsed time: 0.08 min
 >> oof_df created. elapsed time: 0.13 min


,series_id,step,minute,second,class_pred,class_target
0,038441c925bb,3240.0,0,0,0.039498,0.0
1,038441c925bb,3241.0,0,5,NaN,NaN
2,038441c925bb,3242.0,0,10,NaN,NaN
3,038441c925bb,3243.0,0,15,NaN,NaN
4,038441c925bb,3244.0,0,20,NaN,NaN


...

,series_id,step,minute,second,class_pred,class_target
0,05e1944c3818,18000.0,0,0,0.077786,0.0
12,05e1944c3818,18012.0,1,0,0.047924,0.0
24,05e1944c3818,18024.0,2,0,0.020960,0.0
36,05e1944c3818,18036.0,3,0,0.012466,0.0
48,05e1944c3818,18048.0,4,0,0.004096,0.0


save oof_df to  /kaggle/working/_oof/exp012_targetdownsample_epoch1/raw_oof_df_fold3.parquet
-- fold4 inference start --
model loading /kaggle/working/exp012_targetdownsample_epoch1/fold4_model.pth
creating oof_df ... merging oof_df
merge elapsed time: 0.07 min
 >> oof_df created. elapsed time: 0.13 min


,series_id,step,minute,second,class_pred,class_target
0,062cae666e2a,1620.0,0,0,0.050507,0.0
1,062cae666e2a,1621.0,0,5,NaN,NaN
2,062cae666e2a,1622.0,0,10,NaN,NaN
3,062cae666e2a,1623.0,0,15,NaN,NaN
4,062cae666e2a,1624.0,0,20,NaN,NaN


,series_id,step,minute,second,class_pred,class_target
0,062cae666e2a,1620.0,0,0,0.050507,0.0
12,062cae666e2a,1632.0,1,0,0.035750,0.0
24,062cae666e2a,1644.0,2,0,0.065871,0.0
36,062cae666e2a,1656.0,3,0,0.030390,0.0
48,062cae666e2a,1668.0,4,0,0.026511,0.0


save oof_df to  /kaggle/working/_oof/exp012_targetdownsample_epoch1/raw_oof_df_fold4.parquet


In [21]:
oof_df_fold


,series_id,step,minute,second,class_pred,class_target
0,062cae666e2a,1620.0,0,0,0.050507,0.0
1,062cae666e2a,1621.0,0,5,-1.000000,-1.0
2,062cae666e2a,1622.0,0,10,-1.000000,-1.0
3,062cae666e2a,1623.0,0,15,-1.000000,-1.0
4,062cae666e2a,1624.0,0,20,-1.000000,-1.0
...,...,...,...,...,...,...
16581955,fb223ed2278c,160555.0,59,35,-1.000000,-1.0
16581956,fb223ed2278c,160556.0,59,40,-1.000000,-1.0
16581957,fb223ed2278c,160557.0,59,45,-1.000000,-1.0
16581958,fb223ed2278c,160558.0,59,50,-1.000000,-1.0
